<a href="https://colab.research.google.com/github/FrenchFreis/CCMACLRL_EXERCISES_COM222ML/blob/main/EXERCISE9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercise 9: Choosing the best performing model on a dataset

Instructions:

- Use the Dataset File to train your model
- Use the Test File to generate your results
- Use the Sample Submission file to generate the same format
- Use all classification models

Submit your results to:
https://www.kaggle.com/competitions/playground-series-s4e10/overview



In [78]:
import pandas as pd
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder


## Dataset File

In [43]:
dataset_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/train.csv?raw=true'
df = pd.read_csv(dataset_url)

## Test File

In [82]:
test_url = 'https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/test.csv?raw=true'
dt=pd.read_csv(test_url)

In [83]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39098 entries, 0 to 39097
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   id                          39098 non-null  int64  
 1   person_age                  39098 non-null  int64  
 2   person_income               39098 non-null  int64  
 3   person_home_ownership       39098 non-null  object 
 4   person_emp_length           39098 non-null  float64
 5   loan_intent                 39098 non-null  object 
 6   loan_grade                  39098 non-null  object 
 7   loan_amnt                   39098 non-null  int64  
 8   loan_int_rate               39098 non-null  float64
 9   loan_percent_income         39098 non-null  float64
 10  cb_person_default_on_file   39098 non-null  object 
 11  cb_person_cred_hist_length  39098 non-null  int64  
dtypes: float64(3), int64(5), object(4)
memory usage: 3.6+ MB


## Sample Submission File

In [79]:
sample_submission_url ='https://github.com/robitussin/CCMACLRL_EXERCISES/blob/main/datasets/loan_approval/sample_submission.csv?raw=true'

sf=pd.read_csv(sample_submission_url)

In [80]:
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39098 entries, 0 to 39097
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           39098 non-null  int64  
 1   loan_status  39098 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 611.0 KB


In [84]:
label_encoder = LabelEncoder()
df['person_home_ownership'] = label_encoder.fit_transform(df['person_home_ownership'])
dt['person_home_ownership'] = label_encoder.fit_transform(dt['person_home_ownership'])


df['cb_person_default_on_file'] = label_encoder.fit_transform(df['cb_person_default_on_file'])
dt['cb_person_default_on_file'] = label_encoder.fit_transform(dt['cb_person_default_on_file'])


df['loan_intent'] = label_encoder.fit_transform(df['loan_intent'])
dt['loan_intent'] = label_encoder.fit_transform(dt['loan_intent'])


df['loan_grade'] = label_encoder.fit_transform(df['loan_grade'])
dt['loan_grade'] = label_encoder.fit_transform(dt['loan_grade'])


df['cb_person_default_on_file'] = label_encoder.fit_transform(df['cb_person_default_on_file'])
dt['cb_person_default_on_file'] = label_encoder.fit_transform(dt['cb_person_default_on_file'])


df

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,3,0.0,1,1,6000,11.49,0.17,0,14,0
1,1,22,56000,2,6.0,3,2,4000,13.35,0.07,0,2,0
2,2,29,28800,2,8.0,4,0,6000,8.90,0.21,0,10,0
3,3,30,70000,3,14.0,5,1,12000,11.11,0.17,0,5,0
4,4,22,60000,3,2.0,3,0,6000,6.92,0.10,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58640,58640,34,120000,0,5.0,1,3,25000,15.95,0.21,1,10,0
58641,58641,28,28800,3,0.0,3,2,10000,12.73,0.35,0,8,1
58642,58642,23,44000,3,7.0,1,3,6800,16.00,0.15,0,2,1
58643,58643,22,30000,3,2.0,1,0,5000,8.90,0.17,0,3,0


## 1. Train a KNN Classifier

In [49]:
df.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,3,0.0,1,1,6000,11.49,0.17,0,14,0
1,1,22,56000,2,6.0,3,2,4000,13.35,0.07,0,2,0
2,2,29,28800,2,8.0,4,0,6000,8.90,0.21,0,10,0
3,3,30,70000,3,14.0,5,1,12000,11.11,0.17,0,5,0
4,4,22,60000,3,2.0,3,0,6000,6.92,0.10,0,3,0


In [50]:
from sklearn.model_selection import train_test_split
x = df.drop(["loan_status", "id"], axis=1)
y = df["loan_status"]

x_train, x_test, y_train, y_test = train_test_split(x,y,random_state=1,test_size=0.3)

- Perform cross validation

In [51]:
score_list = {}

In [52]:
KNN = KNeighborsClassifier(n_neighbors=22)

KNN.fit(x_train,y_train)
knn_score = KNN.score(x_test,y_test)
score_list["KNN Classifier"] = knn_score
print(f"Score is {knn_score}")

Score is 0.8938274411731272


In [53]:
scores = cross_val_score(KNN, x, y, cv=10)
scores

array([0.89360614, 0.89565217, 0.89479966, 0.89497016, 0.89497016,
       0.89409959, 0.89341746, 0.89665757, 0.89034789, 0.89819236])

In [54]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
score_list["KNN Classifier"] = scores.mean()


0.89 accuracy with a standard deviation of 0.00


## 2. Train a Logistic Regression Classifier

In [55]:
LR = LogisticRegression()
LR.fit(x_train,y_train)
lr_score = LR.score(x_test,y_test)
score_list["LogisticRegression"] = lr_score
print(f"Score is {lr_score}")

Score is 0.8858133454586791


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


- Perform cross validation

In [56]:
scores = cross_val_score(LR, x, y, cv=10)
scores

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

array([0.88422847, 0.88354646, 0.88269395, 0.88235294, 0.8855925 ,
       0.88437926, 0.87994543, 0.88437926, 0.88523192, 0.88420873])

In [57]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
score_list["LogisticRegression"] = scores.mean()

0.88 accuracy with a standard deviation of 0.00


## 3. Train a Naive Bayes Classifier

In [58]:
NBC = GaussianNB()
NBC.fit(x_train,y_train)
nbc_score = NBC.score(x_test,y_test)

print(f"Score is {nbc_score}")

Score is 0.8824599295214277


- Perform cross validation

In [59]:
scores = cross_val_score(NBC, x, y, cv=10)
scores

array([0.88167093, 0.87740835, 0.88269395, 0.87911338, 0.88610401,
       0.87858117, 0.8840382 , 0.88182128, 0.88625512, 0.88062756])

In [60]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
score_list["Naive Bayes"] = scores.mean()

0.88 accuracy with a standard deviation of 0.00


## 4. Train a SVM Classifier

In [61]:
SVM = SVC()
SVM.fit(x_train,y_train)
svm_score = SVM.score(x_test,y_test)
print(f"Score is {svm_score}")

Score is 0.8579061043537569


- Perform cross validation

In [62]:
scores = cross_val_score(SVM, x, y, cv=10)
scores

array([0.85763001, 0.85763001, 0.85745951, 0.85763001, 0.85763001,
       0.85760573, 0.85777626, 0.8574352 , 0.85760573, 0.85760573])

In [63]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
score_list["SVM"] = scores.mean()

0.86 accuracy with a standard deviation of 0.00


## 5. Train a Decision Tree Classifier

In [64]:

DTC = DecisionTreeClassifier()
DTC.fit(x_train,y_train)
dtc_score = DTC.score(x_test,y_test)
print(f"Score is {dtc_score}")

Score is 0.9101398203933159


- Perform cross validation

In [65]:
scores = cross_val_score(DTC, x, y, cv=10)
scores

array([0.91236147, 0.90605286, 0.90980392, 0.91321398, 0.91065644,
       0.90671896, 0.91132333, 0.91797408, 0.91456344, 0.91626876])

In [66]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
score_list["Decision Tree Classification"] = scores.mean()

0.91 accuracy with a standard deviation of 0.00


## 6. Train a Random Forest Classifier

In [67]:
RFC = RandomForestClassifier()
RFC.fit(x_train,y_train)
rfc_score = RFC.score(x_test,y_test)
print(f"Score is {rfc_score}")

Score is 0.9489598726838695


In [68]:
scores = cross_val_score(RFC, x, y, cv=10)
scores

array([0.94697357, 0.94492754, 0.94901961, 0.95038363, 0.95021313,
       0.94901091, 0.94969304, 0.95327422, 0.94866985, 0.9505457 ])

In [69]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))
score_list["Random Forest Classification"] = scores.mean()

0.95 accuracy with a standard deviation of 0.00


## 7. Compare all the performance of all classification models

In [70]:
score_list = list(score_list.items())


In [71]:
for alg,score in score_list:
    print(f"{alg} Score is {str(score)[:4]} ")

KNN Classifier Score is 0.89 
LogisticRegression Score is 0.88 
Naive Bayes Score is 0.88 
SVM Score is 0.85 
Decision Tree Classification Score is 0.91 
Random Forest Classification Score is 0.94 


In [85]:
dt.drop(["id"],axis=1,inplace=True)

## 9. Generate Submission File

Choose the model that has the best performance to generate a submission file.

In [81]:
id = sf.pop('id')
y_pred = RFC.predict(dt)

# Create a submission DataFrame
submission_df = pd.DataFrame({
    'id': id,
    'loan_status': y_pred
})

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission_file.csv', index=False)
print("Submission file created: submission_file.csv")

Submission file created: submission_file.csv
